# Deep Q Network for different environments

## 1. Import the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt #plotting library
from matplotlib import animation #animated visualizations library
from collections import namedtuple, deque 
#nametuple creates tuple subclasses with name fields, access elements by names instead of index
#deque (double-ended queue) for adding and removing elements from both ends
from tqdm import tqdm
#add progress bars to Python code for easy monitoring progress of loops and tasks
# %matplotlib inline 
import gym #environments for agents
from datetime import datetime #manipulating dates and times
import pandas as pd #work with structured data
import torch #Pytorch supports tensor computations and neural networks
import torch.nn as nn #Pytorch supports building neural networks
import torch.nn.functional as Function 
#common functions in neural network operations 
    # Activation functions (ReLU, sigmoid, tanh)
    # Loss functions (cross_entropy, mse_loss)
    #Utility functions for tensor manipulation (softmax, dropout, batch_norm, etc.)
import torch.optim as optim #optimization algorithms for training neural networks
import random #generate random numbers/selections
from collections import namedtuple, deque 
import itertools 
# provides various functions for creating iterators and combining them for complex interators
# includes cycle, chain, zip, etc.

## 2. Neural network model for approximating Q-values 

In [ ]:
class QNetwork(nn.Module):
    #Actor Model

    def __init__(self, state_size, action_size, seed, fc1_units=10, fc2_units=10):
        #Parameters
#             state_size (int): Dimensionality of input state space
#             action_size (int): Dimensionality of output action space
#             seed (int): Random seed for reproducibility
#             fc1_units (int): Number of neurons (units) in first fully connected hidden layer
#             fc2_units (int): Number of neurons (unit) in second fully connected hidden layer  
        
        #Initialization
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed) # Set the random seed :
        state_size = env.observation_space.shape[0]
        #Ensure random numbers generate are reproducible
        #Running same code with the same seed will produce the same sequence of random numbers
        
        # nn.Linear creates fully connected layers (input units, output units)
        self.fc1 = nn.Linear(state_size, fc1_units) 
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        # Network to map state with action values (Q-values)
        #inside method, input state pass through each layer
        #ReLU activation functions are applied to the outputs of hidden layers
#         print('state:', state.shape)
        #x output of 1 layer is input to the next
        x = Function.relu(self.fc1(state))
        x = Function.relu(self.fc2(x))
        return self.fc3(x)

## 3. Implement the replay buffer

In [ ]:
device = torch.device("cpu") #Device Initialization

class ReplayBuffer: #Fixed-size buffer to store experience tuples.
    # Initialize a ReplayBuffer object.
    def __init__(self, state_size, action_size, buffer_size, batch_size, priority=False):
            #state_size(int): dimension of each state
            #action_size (int): dimension of each action 
            #buffer_size (int): max size of buffer 
            #batch_size (int): size of 1 training batch
            #seed (int): random seed
                
        #Pointer to keep track of position within the replay buffer where next experiene will be added
        # ptr = 0, new experience added -> increment to ptr = 1
        self. ptr = 0
        #Check if buffer have been filled
        self.n = 0 
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.state_size = state_size
        
        state_size = env.observation_space.shape[0]

        #create states tensor full of zeros with {buffer_size} rows & {state_size} columns
        #store in device CPU, allocate RAM to it 
        self.states = torch.zeros((buffer_size,)+(state_size,)).to(device)
        
        #Similar to above 
        self.next_states = torch.zeros((buffer_size,)+(state_size,)).to(device)
        
        #Only 1 action taken per experience tuple stored
        # => action size = buffer size rows but 1 column only needed to store
        self.actions = torch.zeros(buffer_size,1, dtype=torch.long).to(device)
        
        # Reward same with action
        self.rewards = torch.zeros(buffer_size, 1, dtype=torch.float).to(device)
        
        # Flag to indicate transition/end between an episode ('done' flag)
        # True/False floating values (0.0 False, 1.0 True)
        self.dones = torch.zeros(buffer_size, 1, dtype=torch.float).to(device) 
        
        # Error in case implement prioritize replay
        self.error = np.zeros((buffer_size, 1), dtype=float)
        
        # Priority
        self.priority = priority
    
  # Add new experience to buffer
    def add(self, state, action, reward, next_state, done, state_size):
        
        state_size = env.observation_space.shape[0]
        
        
        # Function to flatten ragged nested sequences into a flat array of compatible type
        def flatten_nested_sequence(elem):
            flattened = []
            for e in elem:
                if isinstance(e, (list, tuple)):
                    flattened.extend(flatten_nested_sequence(e))
                elif isinstance(e, np.ndarray):
                    if e.dtype == np.object_:
                        raise ValueError("Cannot handle np.ndarray of type numpy.object_")
                    else:
                        flattened.append(e)
                else:
                    flattened.append(e)
            return flattened

        # Convert state and next_state to appropriate types
        if isinstance(state, np.ndarray):
            state = flatten_nested_sequence(state)
            state = torch.as_tensor(state, dtype=torch.float32).to(device)
        elif isinstance(state, torch.Tensor):
            state = state.to(device)
        else:
            state = torch.zeros((1, state_size), dtype=torch.float32).to(device)

        if isinstance(next_state, np.ndarray):
            next_state = flatten_nested_sequence(next_state)
            next_state = torch.as_tensor(next_state, dtype=torch.float32).to(device)
        elif isinstance(next_state, torch.Tensor):
            next_state = next_state.to(device)
        else:
            next_state = torch.zeros((1, state_size), dtype=torch.float32).to(device) 
        # Convert action, reward, and done flag to tensors
        action = torch.as_tensor(action, dtype=torch.float32).to(device)
        reward = torch.as_tensor(reward, dtype=torch.float32).to(device)
        done = torch.as_tensor(done, dtype=torch.bool).to(device)

        # Store the data in the replay buffer
        self.states[self.ptr] = state
        self.next_states[self.ptr] = next_state
        self.actions[self.ptr] = action
        self.rewards[self.ptr] = reward
        self.dones[self.ptr] = done

        # Increment the pointer
        self.ptr = (self.ptr + 1) % self.buffer_size

        # Reset pointer and flag when buffer is filled
        if self.ptr == 0:
            self.n = self.buffer_size
        
    #Sample a batch of experience from memory
    def sample(self, get_all=False):
        n = len(self) # Length of the Buffer
        
        # Return all experience stored in buffer, no sampling
        if get_all:
            return self.states[:n], self.actions[:n], self.rewards[:n], self.next_states[:n], self.dones[:n]
        
        # else, do sampling: 
        else:
            if self.priority:     
            #enable prioritized experience replay
            #experience are sampled based on priorities probability distribution p = self.error
                idx = np.random.choice(n, self.batch_size, replace=False, p=self.error)
            else: #uniform sampling
                idx = np.random.choice(n, self.batch_size, replace=False)
            #Replace = False to only sample once for each element
            
            #Retrieve sampled experiences 
            states = self.states[idx]
            next_states = self.next_states[idx]
            actions = self.actions[idx]
            rewards = self.rewards[idx]
            dones = self.dones[idx]
        
            return (states, actions, rewards, next_states, dones), idx
    
    # Update the error associated with experiences in replay buffer
    def update_error(self, error, idx=None): #specify index number, if not specify, all are updated
        error = torch.abs(error.detach()) #absolute value of errors, detach to prevent gradient computation to be attached to error tensor
        error = error / error.sum() #Normalize, ensure all errors add up to 1
        if idx is not None: #index are specified, then only update specified indices
            self.error[idx] = error.cpu().numpy()
        else: # not specify index, all are updated
            self.error[:len(self)] = error.cpu().numpy()
    
    # 
    def __len__(self): 
        if self.n == 0:
            return self.ptr
        else:
            return self.n #when buffer is filled self.n stored that size

## 4. Set the parameters 

In [ ]:
buffer_size = 100000 #Replay buffer size
batch_size = 4 #Minibatch size
gamma = 0.99 #Discount factor
tau = 1 #Soft update target parameters, tau = 1 = copy completely
alpha = 0.0005 #learning rate
update_every = 4 #How often to update the network

## 5. Agent learning implementation

In [ ]:
class Agent(): #Learning by interacting with the environment
    def __init__(self, state_size, action_size, seed=1, ddqn=True, priority=False):
        self.state_size = state_size #Dimension of each state
        self.action_size = action_size #Dimension of action
        self.seed = random.seed(seed) #Choose the random seed
        self.ddqn = ddqn #Store whether agent uses DDQN
        self.priority = priority #Whether uses experience replay
        
        #Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        
        #Initializes optimizer for updating the weights of local Q-Network
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=alpha)

        # Replay memory
        self.memory = ReplayBuffer(state_size, (action_size,), buffer_size, batch_size)
        # Initialize timestep (to keep track timesteps for updating target)
        self.t_step = 0
        
    def step(self, state, action, reward, next_state, done, step_size):
        
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done, step_size)
        
        
        self.t_step = self.t_step + 1  
        ## Update Q-network after each step in DQN case
        
        if not self.ddqn: #DQN case
            
            if len(self.memory) > batch_size:
                experiences, idx = self.memory.sample() #sample batch of experience from replay memory
                error = self.learn(experiences) #update Q-network based on sampled experiences
                                #outputs the error (difference) between predicted Q-value and target Q-value during training
                self.memory.update_error(error,idx)
                
        else: #DDQN case, only update after a period
            if len(self.memory) > batch_size:
                experiences, idx = self.memory.sample() #sample batch of experience from replay memory
                error = self.learn(experiences) #update Q-network based on sampled experiences
                                #outputs the error (difference) between predicted Q-value and target Q-value during training
            #if there is no remainder we update
            if (self.t_step % update_every) == 0 and len(self.memory) > batch_size:
                self.memory.update_error(error, idx) #update
 
    def act(self, state, epsilon):
        if isinstance(state, np.ndarray):
            if state.dtype == np.object_:
                # Convert elements of the NumPy array to compatible types
                state = np.array([self.convert_to_compatible_type(elem) for elem in state])
            # Convert the modified NumPy array to a PyTorch tensor for neural network input
            state = torch.as_tensor(state, dtype=torch.float32).to(device)
        elif isinstance(state, list) or isinstance(state, tuple):
            # Convert the state to a NumPy array and handle ragged nested sequences
            state = self.handle_ragged_nested_sequence(state)
            # Convert the modified NumPy array to a PyTorch tensor for neural network input
            state = torch.as_tensor(state, dtype=torch.float32).to(device)
        elif isinstance(state, torch.Tensor):
            # Ensure the tensor is in the correct device and data type
            state = state.to(device)
        else:
            raise TypeError("Input state should be a NumPy array, list, tuple, or torch.Tensor.")

        # Ensure the state has the correct shape for the neural network input
        state = state.unsqueeze(0)  # Add a batch dimension

        self.qnetwork_local.eval()  # Evaluation mode
        with torch.no_grad():  # Disable gradient calculation when choosing action
            action_values = self.qnetwork_local(state)  # Pass preprocessed states into local Q-network
        self.qnetwork_local.train()  # Set local Q network back to training mode after inference is complete

        # Epsilon-greedy action selection
        if random.random() > epsilon:
            # Return action with highest Q-value
            action = np.argmax(action_values.cpu().data.numpy())
        else:
            # Select a random action
            action = random.choice(range(self.action_size))
        action = min(max(action, 0), self.action_size - 1) # Ensure action is within valid range
        
        return action

    def convert_to_compatible_type(self, elem):
        # Convert the element to a compatible type if it is a numpy.object_
        if isinstance(elem, str):
            return float(elem)
        elif isinstance(elem, np.ndarray):
            return elem.astype(np.float32)
        else:
            return float(elem)

    def handle_ragged_nested_sequence(self, state):
        if not state:
            return np.array([])  # Return an empty NumPy array if the state is empty
        else:
            # Convert dictionaries to lists of values
            state = [list(d.values()) if isinstance(d, dict) else d for d in state]
            # Determine the maximum length among all nested sequences
            max_length = max(len(sublist) for sublist in state)
            # Pad shorter sequences with zeros to make them of equal length
            state = [sublist + [0] * (max_length - len(sublist)) if len(sublist) < max_length else sublist for sublist in state]
            # Convert the nested sequence to a NumPy array
            state = np.array(state)
            return state
    
    #Error update for prioritization experience replay
    def update_error(self):
        #Sample experience from replay memory
        states, actions, rewards, next_states, dones = self.memory.sample(get_all=True)
        with torch.no_grad(): #no gradient computations within block
            if self.ddqn: #DDQN is enabled, use Q Network for both local and target
                #Q values of action taken in current state
                old_val = self.qnetwork_local(states).gather(-1, actions)
                #Determine actions with highest Q values in next states using local Q-network
                actions = self.qnetwork_local(next_states).argmax(-1, keepdim=True)
                # Compute Q-values of selected actions from target Q-network for next states
                maxQ = self.qnetwork_target(next_states).gather(-1, actions)
                #Calculate target as the normal formula
                #dones is a binary tensor indicating the next state is terminal or not (store done flags)
                #Use 1-dones to ensure discounted future rewards are considered only for non-terminal states
                #At terminal done = 1, so 1 - dones = 0, cancels out the maxQ => Q terminal = 0
                target = rewards+gamma*maxQ*(1-dones)
            else: # Normal DQN
                #Difference in this DQN is obtained directly from target Q-network using max function
                #instead of considering the action from local state
                maxQ = self.qnetwork_target(next_states).max(-1, keepdim=True)[0]
                target = rewards+gamma*maxQ*(1-dones)
                old_val = self.qnetwork_local(states).gather(-1, actions)
            error = old_val - target
            self.memory.update_error(error)      
            
    #Update value parameters using batch of experience tuples
    def learn(self, experiences):
        #experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
        #gamma (float): discount factor
      
        states, actions, rewards, next_states, dones = experiences

        ## compute and minimize the loss
        self.optimizer.zero_grad() #Resets all gradients to zero, gradients from previous batches do not accumulate
        if i_episode < 3 or i_episode > 1998:
            print("Current state: ", state)
            print(" ")
            print("Experiences: ")
            print("States: ", states)
            print("Actions: ", actions)
            print("Rewards: ", rewards)
            print("Next states: ", next_states)
            print(" ")
        #Same as explained above
        if self.ddqn: #DDQN
            old_val = self.qnetwork_local(states).gather(-1, actions)
            if i_episode < 3 or i_episode > 1998:
                print("Q_current: ", old_val)
                print(" ")
            with torch.no_grad():
                if i_episode < 3 or i_episode > 1998:
                    print("All Q values for next_states in order to choose actions, estimate with local network: ")
                    print(self.qnetwork_local(next_states))
                    print(" ")
                next_actions = self.qnetwork_local(next_states).argmax(-1, keepdim=True)
                
                if i_episode < 3 or i_episode > 1998:
                    print("Actions, chose with local network = ", next_actions)
                    print(" ")
                    print("All Q values for next_states, estimate with target network: ")
                    print(self.qnetwork_target(next_states))
                    print(" ")
                maxQ = self.qnetwork_target(next_states).gather(-1, next_actions)
                
                if i_episode < 3 or i_episode > 1998:
                    print("Chosen maximum Q - value from actions gathered above = ", maxQ)
                    print(" ")
                target = rewards+gamma*maxQ*(1-dones)
                
                if i_episode < 3 or i_episode > 1998:
                    print("Q_target = ", target)
                    print(" ")
                    
        else: # Normal DQN
            old_val = self.qnetwork_local(states).gather(-1, actions)
            
            if i_episode < 3 or i_episode > 1998:
                print("Q_current: ", old_val)
                print(" ")
                print("Actions = ", actions)
                print(" ")
            
            with torch.no_grad():
                
                if i_episode < 3 or i_episode > 1998:
                    print("All Q values for next_states = ")
                    print(self.qnetwork_target(next_states))
                    print(" ")
                
                maxQ = self.qnetwork_target(next_states).max(-1, keepdim=True)[0]
                
                if i_episode < 3 or i_episode > 1998:
                    print("Chosen maximum Q - value= ", maxQ)
                    print(" ")
                
                target = rewards+gamma*maxQ*(1-dones)
                
                if i_episode < 3 or i_episode > 1998:
                    print("Q_target = ", target)
                    print(" ")
               
        if i_episode < 3 or i_episode > 1998:
            print("Error(Q_target - Q_current): ", target - old_val)
            print(" ")
        loss = Function.mse_loss(old_val, target) #Calculate loss of current and target Q-values
        loss.backward() #Gradient of loss with respect to Q-network parameters are computed
                        #and used to update the parameters
        self.optimizer.step() #update the neural network
                                #step method applies optimization to update parameters
                                #steps:
                                #1. Optimizer uses computed gradients to update parameters
                                #2. Adam optimizer is applied to each parameter based on the gradients and learning rate
                                #3. Gradients are cleared back to zero as first step for next training iteration

        ##UPDATE THE TARGET NETWORK##
        if not self.ddqn:  # If DQN, update after each episode
            self.soft_update(self.qnetwork_local, self.qnetwork_target, tau) 
        else:  # If DDQN, update after every update_every episodes
            if (self.t_step % update_every) == 0:
                self.soft_update(self.qnetwork_local, self.qnetwork_target, tau)
        
        return old_val - target #temporal difference TD error between old Q and target Q to monitor training progress

    
    def soft_update(self, local_model, target_model, tau):
        #local_model: online model, actively being trained. Weights will be copied from here
        #target_model: use to generate target Q-values during training. Weights will be copied to here
        #tau: interpolation parameters determine rate at which parameters of target models are updated
        #small tau slower update, big tau faster update, less stable
        
        #function iterates over parameters of both target model and local model using zip
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            #for each target param - local param pair, update target param by the formula
            # target_param = tau*local_param + (1-tau)*target_local 
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
            
    # Function to simulate a model in an environment
    def simulate_model(env_name, model_path):
        # Load the environment
        env = gym.make(env_name)

        # Get environment parameters
        state_size = env.observation_space.shape[0]
        action_size = env.action_space.n

        # Initialize the agent
        agent = Agent(state_size, action_size, seed) 

        # Load the model weights
        agent.qnetwork_local.load_state_dict(torch.load(model_path))
        agent.qnetwork_local.eval()

        # Simulate the model in the environment
        scores = []
        n_episodes = 100  # Number of episodes for simulation
        max_t = 1000  # Maximum number of timesteps per episode

        for i_episode in tqdm(range(1, n_episodes+1)):
            state = env.reset()
            score = 0

            for t in range(max_t):
                action = agent.act(state, epsilon=0.)  # Greedy action selection, no exploration
                state, reward, done, _ = env.step(action)
                score += reward
                if done:
                    break

            scores.append(score)

        # Close the environment
        env.close()

        # Print average score
        print("Average score:", np.mean(scores))

## 6. Training parameters and environments

In [ ]:
envs = ['LunarLander-v2'] #list of environments 'MountainCar-v0',
#list of implementing algos, each element consists of [DDQN is enabled?, Prioritized experience replay enabled?, type of algo]
algos = [[True, False, 'DDQN']] #,, [True, True, 'PriorityDDQN'], [False, False, 'DQN'], 
seed = 1
n_episodes = 2000 #number of training episodes
max_t = 1000 #maximum number of timesteps
epsilon_start = 0.7 #starting value of epsilon greedy
epsilon_end = 0.01 #minimum value of epsilon
epsilon_decay = 0.995 #rate at which epsilon decays

## 7. Training implementation

In [ ]:
print("Seed = 1")
for i in envs:
    print("ENVIRONMENT:-----------", i)
    env = gym.make(i)
    res=[]
    for j in algos:
        print("Algorithm:", j[2])
        rewards = []
        aver_reward = []
        aver = deque(maxlen=100)
        state_size = env.observation_space.shape[0]
        action_size=env.action_space.n
        agent = Agent(state_size, action_size, seed, ddqn=j[0], priority=j[1])
        epsilon = epsilon_start                    # initialize epsilon
        
        for i_episode in tqdm(range(1, n_episodes+1)):
            if i_episode < 3 or i_episode > 1998:
                print("Iteration number: ", i_episode-1)
            state = env.reset()
            score = 0
            for t in range(max_t):
                action = agent.act(state, epsilon)
                step_result = env.step(action)
                next_state, reward, done, _ = step_result[:4]
                agent.step(state, action, reward, next_state, done, state_size)
                state = next_state
                score = score + reward
                if done:
                    break 

            aver.append(score)     
            aver_reward.append(np.mean(aver))
            rewards.append(score)
            epsilon = max(epsilon_end, epsilon_decay*epsilon) # decrease epsilon
            
        reward="model/"+i+"_"+j[2]+"_"+str(n_episodes)+"_"+str(datetime.now().strftime("%Y%m%d%H%M%S"))
        torch.save(agent.qnetwork_local.state_dict(),reward+'.pt')
        res.append(aver_reward)
        print("----------------End Algorithm--------------------")
    
    fig=plt.figure()   
    
    reward='plots/'+i+'_result'+str(datetime.now().strftime("%Y%m%d%H%M%S"))
    df=pd.DataFrame({'DQN':res[0],'DDQN':res[1],'PriorityDDQN':res[2]})
    df.to_csv(reward+'.csv')
    print("------------------------End Environment-------------------")
    
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.plot(df['DQN'], 'r', label='DQN')

    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.plot(df['DDQN'], 'orange',label='DDQN')

    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.plot(df['PriorityDDQN'],'b',label='PER')
    
    plt.title('Learning Curve '+i)

    #Insert the legends in the plot
    fig.legend(loc='lower right')
    fig.savefig(reward+'.png', dpi=100)
    
        

## 8. Demonstration with random policy

In [ ]:
# Load the environment
env_name = 'MountainCar-v0'  # Change this to the environment you want to simulate
env = gym.make(env_name, render_mode = 'human')

# Get environment parameters
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Initialize the agent
agent = Agent(state_size, action_size, seed=65)  

# Simulate the model in the environment with random actions
scores = []
n_episodes = 100  # Number of episodes for simulation
max_t = 1000  # Maximum number of timesteps per episode

for i_episode in tqdm(range(1, n_episodes+1)):
    state = env.reset()
    env.render()
    score = 0
    
    for t in range(max_t):
        action = agent.act(state, epsilon=0.5) 
        step_result = env.step(action)
        next_state, reward, done, _ = step_result[:4]
        state = next_state
        score = score + reward
        if done:
            break
    print(score)        
    scores.append(score)

# Close the environment
env.close()

# Print average score
print("Average score with random actions:", np.mean(scores))

## 9. Demonstration with learned policy

In [ ]:
# Load the environment
env_name = 'MountainCar-v0'  # Change this to the environment you want to simulate
env = gym.make(env_name, render_mode = 'human')

# Get environment parameters
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Initialize the agent
agent = Agent(state_size, action_size, seed=1)  
# Load the model weights
model_path = 'model/Seed42_MountainCar-v0_PriorityDDQN_4000_20240404234612.pt'  # Path to your trained model
agent.qnetwork_local.load_state_dict(torch.load(model_path))
agent.qnetwork_local.eval()

# Simulate the model in the environment
scores = []
n_episodes = 100  # Number of episodes for simulation
max_t = 1000  # Maximum number of timesteps per episode

for i_episode in tqdm(range(1, n_episodes+1)):
    state = env.reset()
    env.render()
    score = 0
    
    for t in range(max_t):
        action = agent.act(state, epsilon=0.)  # Greedy action selection, no exploration
        step_result = env.step(action)
        next_state, reward, done, _ = step_result[:4]
        state = next_state
        score = score + reward
        if done:
            break
    print(score)        
    scores.append(score)

# Close the environment
env.close()

# Print average score
print("Average score:", np.mean(scores))

## 10. Draw the shaded plot for each case

In [ ]:
# Step 1: Read Data from CSV
df = pd.read_csv('plots/DQN_LunarLander-v2_reward.csv')

# Step 2: Remove the first row (Episodes, Run 1, Run 2, Run 3) to keep only the rewards data
# df = df.drop(0)

# Step 3: Convert the remaining DataFrame to numeric values
df = df.apply(pd.to_numeric)

# Step 4: Select only the three late columns (Run 1, Run 2, Run 3)
later_columns = df.iloc[:, 1:4]

# Step 5: Calculate Mean and Standard Deviation for the three later columns
mean_values = later_columns.mean(axis=1)
std_values = later_columns.std(axis=1)
print("Standard deviation: ", std_values)
print("Mean: ", mean_values)

# Step 6: Plot the Data
plt.figure(figsize=(12, 6))
plt.plot(mean_values, label='Mean Sum of Rewards', color = "red")
plt.fill_between(mean_values.index, mean_values - std_values, mean_values + std_values, alpha=0.5, label='Standard Deviation', color = "orange")
plt.xlabel('Episode')
plt.ylabel('Sum of Rewards')
plt.title('Shaded Plot of Sum of Rewards with Mean and Standard Deviation for Deep Q-Learning Algorithm, Lunar Lander')
plt.savefig('plots/shaded_plot_DQN_LunarLander.png')
plt.legend()
plt.show()